### <center><p style="color:orange">Handwritten digit recognition</p></center>

#### Importing Libraries

In [47]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image, ImageOps
from scipy.ndimage import center_of_mass, shift

#### Load Dataset (CSV File)

In [48]:
data = pd.read_csv("./train.csv")
data = np.array(data)
np.random.shuffle(data)

#### Split the data into Testing and Training part

In [49]:
# Split test
data_dev = data[:1000].T
Y_dev = data_dev[0]
X_dev = data_dev[1:] / 255.0

# Split train
data_train = data[1000:].T
Y_train = data_train[0]
X_train = data_train[1:] / 255.0
_, m_train = X_train.shape

In [50]:
def initialize_parameters():
    """
    Initialize weights and biases for a 2-layer neural network.

    Returns:
        W1 (ndarray): Weight matrix for layer 1 of shape (10, 784)
        b1 (ndarray): Bias vector for layer 1 of shape (10, 1)
        W2 (ndarray): Weight matrix for layer 2 of shape (10, 10)
        b2 (ndarray): Bias vector for layer 2 of shape (10, 1)
    """
    W1 = np.random.randn(10, 784) * 0.01
    b1 = np.zeros((10, 1))
    W2 = np.random.randn(10, 10) * 0.01
    b2 = np.zeros((10, 1))
    return W1, b1, W2, b2


def ReLU(Z):
    """
    Apply ReLU activation function element-wise.

    Args:
        Z (ndarray): Input array.

    Returns:
        ndarray: Output after applying ReLU.
    """
    return np.maximum(Z, 0)


def relu_derivative(Z):
    """
    Compute derivative of ReLU activation.

    Args:
        Z (ndarray): Input array.

    Returns:
        ndarray: Derivative of ReLU.
    """
    return Z > 0


def softmax(Z):
    """
    Apply softmax function for multi-class classification.

    Args:
        Z (ndarray): Input array of shape (num_classes, num_samples).

    Returns:
        ndarray: Softmax probabilities.
    """
    expZ = np.exp(Z - np.max(Z, axis=0)) 
    return expZ / np.sum(expZ, axis=0, keepdims=True)


def forward_propagation(W1, b1, W2, b2, X):
    """
    Perform forward propagation through the 2-layer neural network.

    Args:
        W1, b1: Parameters for first layer
        W2, b2: Parameters for second layer
        X (ndarray): Input data of shape (784, m)

    Returns:
        Z1, A1, Z2, A2: Intermediate computations
    """
    Z1 = W1 @ X + b1
    A1 = ReLU(Z1)
    Z2 = W2 @ A1 + b2
    A2 = softmax(Z2)
    return Z1, A1, Z2, A2


def one_hot_encode(Y):
    """
    One-hot encode label vector Y.

    Args:
        Y (ndarray): Label vector of shape (m,)

    Returns:
        one_hot_Y (ndarray): One-hot encoded matrix of shape (num_classes, m)
    """
    one_hot_Y = np.zeros((Y.max() + 1, Y.size))
    one_hot_Y[Y, np.arange(Y.size)] = 1
    return one_hot_Y


def backward_propagation(Z1, A1, Z2, A2, W2, X, Y):
    """
    Perform backward propagation to compute gradients.

    Args:
        Z1, A1, Z2, A2: Values from forward pass
        W2 (ndarray): Weight matrix of layer 2
        X (ndarray): Input data
        Y (ndarray): True labels

    Returns:
        Gradients dW1, db1, dW2, db2
    """
    m = Y.size
    one_hot_Y = one_hot_encode(Y)

    dZ2 = A2 - one_hot_Y
    dW2 = (1 / m) * dZ2 @ A1.T
    db2 = (1 / m) * np.sum(dZ2, axis=1, keepdims=True)

    dZ1 = W2.T @ dZ2 * relu_derivative(Z1)
    dW1 = (1 / m) * dZ1 @ X.T
    db1 = (1 / m) * np.sum(dZ1, axis=1, keepdims=True)

    return dW1, db1, dW2, db2


def update_parameters(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha):
    """
    Update weights and biases using gradient descent.

    Args:
        W1, b1, W2, b2: Current parameters
        dW1, db1, dW2, db2: Gradients
        alpha (float): Learning rate

    Returns:
        Updated W1, b1, W2, b2
    """
    W1 -= alpha * dW1
    b1 -= alpha * db1
    W2 -= alpha * dW2
    b2 -= alpha * db2
    return W1, b1, W2, b2


def get_predictions(A2):
    """
    Get predicted class labels from output layer.

    Args:
        A2 (ndarray): Softmax probabilities of shape (10, m)

    Returns:
        ndarray: Predicted labels of shape (m,)
    """
    return np.argmax(A2, axis=0)


def get_accuracy(predictions, Y):
    """
    Calculate accuracy between predictions and true labels.

    Args:
        predictions (ndarray): Predicted labels
        Y (ndarray): True labels

    Returns:
        float: Accuracy score between 0 and 1
    """
    return np.mean(predictions == Y)

#### Training model function

In [51]:
def train_model(X, Y, alpha=0.1, iterations=500):
    W1, b1, W2, b2 = initialize_parameters()
    for i in range(iterations):
        Z1, A1, Z2, A2 = forward_propagation(W1, b1, W2, b2, X)
        dW1, db1, dW2, db2 = backward_propagation(Z1, A1, Z2, A2, W2, X, Y)
        W1, b1, W2, b2 = update_parameters(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha)
        if i % 50 == 0:
            predictions = get_predictions(A2)
            accuracy = get_accuracy(predictions, Y)
            print(f"Iteration {i}: Accuracy = {accuracy * 100:.2f}%")
    return W1, b1, W2, b2


#### training the Model 

In [52]:
W1, b1, W2, b2 = train_model(X_train, Y_train, alpha=0.1, iterations=1000) # training the model 


Iteration 0: Accuracy = 9.23%
Iteration 50: Accuracy = 22.90%
Iteration 100: Accuracy = 58.00%
Iteration 150: Accuracy = 74.88%
Iteration 200: Accuracy = 81.23%
Iteration 250: Accuracy = 84.60%
Iteration 300: Accuracy = 86.50%
Iteration 350: Accuracy = 87.66%
Iteration 400: Accuracy = 88.44%
Iteration 450: Accuracy = 88.95%
Iteration 500: Accuracy = 89.40%
Iteration 550: Accuracy = 89.70%
Iteration 600: Accuracy = 89.99%
Iteration 650: Accuracy = 90.25%
Iteration 700: Accuracy = 90.50%
Iteration 750: Accuracy = 90.71%
Iteration 800: Accuracy = 90.93%
Iteration 850: Accuracy = 91.10%
Iteration 900: Accuracy = 91.21%
Iteration 950: Accuracy = 91.33%


#### Preprocessing image

In [53]:
def preprocess_image_from_path(image_path):
    image = Image.open(image_path).convert("L")  # converting into gray scale
    image = ImageOps.invert(image)
    image = image.resize((28, 28))
    img_array = np.array(image).astype(np.float32) / 255.0

    cy, cx = center_of_mass(img_array)
    shift_y = 14 - cy
    shift_x = 14 - cx
    img_array = shift(img_array, shift=(shift_y, shift_x), mode='constant', cval=0.0)

    return img_array.reshape(784, 1)

def predict_digit_from_image(image_path):
    X_input = preprocess_image_from_path(image_path)
    _, _, _, A2 = forward_propagation(W1, b1, W2, b2, X_input)
    prediction = get_predictions(A2)[0]
    print(f"We found digit {prediction}")


#### Testing Model

In [54]:
predict_digit_from_image("./Testing_data/digit_0.png")


We found digit 0
